# Simulating Metabolism for CGGA Validation Data

### Using Preferred iMAT params (determined from TCGA CV) - corresponds to Dataset 8

Downloading Model

In [ ]:
from pyGSLModel import download_GSL_model

model = download_GSL_model()

print(f"Number of Reactions in model : {len(model.reactions)}")
print(f"Number of Metabolites in model : {len(model.metabolites)}")
print(f"Number of Genes in model : {len(model.genes)}")

print(f"Checking gene symbol conversion :")
model.genes.get_by_id("UGT8")

Performing simulation against each row in the CGGA dataset

In [ ]:
import pandas as pd
CGGA_df = pd.read_csv("./CGGA_Data/CGGA_LGG_TPM.csv").set_index("CGGA_ID")
TCGA_df = pd.read_csv("./TCGA_Data/TCGA_LGG.csv").set_index("sample")

In [ ]:
# Veryifying input columns are identical between training TCGA data and CGGA validation data
TCGA_df.columns.to_list() == CGGA_df.columns.to_list()

In [ ]:
CGGA_input = CGGA_df.drop(columns=['OS.time', 'OS']).T.copy()
CGGA_input.head()

| Simulation # | Upper Quartile | Lower Quartile | Epsilon | Threshold |
| --- | --- | --- | --- | --- |
| 20 | 0.2 | 0.8 | 100 | 10 | 

In [ ]:
from pyGSLModel import iMAT_multi_integrate
CGGA_simualted = iMAT_multi_integrate(model=model, data=CGGA_input, upper_quantile=0.2, lower_quantile=0.8, epsilon=100, threshold=10)

In [ ]:
CGGA_simulated_df = CGGA_simualted[0]
CGGA_simulated_df.head()

In [ ]:
CGGA_Surv = CGGA_df[['OS', 'OS.time']].copy()
CGGA_Surv = CGGA_Surv
CGGA_Surv.head()

In [ ]:
CGGA_Tidied_Data = pd.merge(CGGA_input.T,CGGA_simulated_df,left_index=True,right_index=True)
CGGA_Tidied_Data = pd.merge(CGGA_Surv, CGGA_Tidied_Data, right_index=True, left_index=True)

Checking validation CGGA data matches columns for TCGA training data

In [ ]:
training_data = pd.read_csv("./iMAT_integrated_data/TCGA_iMAT_integrated_df_20.csv").set_index("sample")


In [ ]:
CGGA_Tidied_Data.columns.to_list() == training_data.columns.to_list()

Saving CGGA Tidied Data

In [ ]:
CGGA_Tidied_Data.to_csv("./CGGA_Data/CGGA_Tidied_Integrated.csv")